In [1]:
import numpy as np
from numpy import sin, sqrt, abs
from numpy.random import randint, rand, choice

In [2]:
def rastrigin(x):
    # -5.12 <= x_i <= 5.12 for i=1,..,n
    # global minimum at f(0,..,0)=0
    if isinstance(x, list):
        x = np.array(x)
    A, n = 10, len(x)
    return A * n + np.sum(x ** 2 - A * np.cos(2 * np.pi * x), axis=0)


def booth(arg):
    # -10 <= x,y <= 10 
    # global minimum at f(1,3)=0
    x, y = arg
    return (x + 2 * y - 7) ** 2 + (2 * x + y - 5) ** 2


def eggholder(arg):
    # -512 <= x,y <= 512
    # global minimum at f(512,404.2319)=-959.6407
    x, y = arg
    return -(y + 47) * sin(sqrt(abs(x / 2 + y + 47))) - x * sin(sqrt(abs(x-y-47)))


functions_data = {
    'rastrigin': {'func': rastrigin, 'bounds': [[-5.12, 5.12], [-5.12, 5.12]], "best": ([0.0, 0.0], 0.0)},
    'booth'    : {'func': booth,     'bounds': [[-10, 10], [-10, 10]], "best": ([1, 3], 0.0)},
    'eggholder': {'func': eggholder, 'bounds': [[-512.0, 512.0], [-512.0, 512.0]], "best": ([512, 404.2319], -959.6407)},
}

In [3]:
def generate_points(obj_func, bounds, k):
    points = np.random.uniform(size=(k, len(bounds)))
    scalers = [right - left for left, right in bounds], [left for left, right in bounds]
    scaled_points = points * scalers[0] + scalers[1]
    return sorted(scaled_points, key=obj_func)   # sort points in acsending order 

# Evolution Strategies

In [4]:
def get_offsprings(pop, sigma, k, bounds):
    offsprings = []
    for p in pop:
        i = 0
        while i < k: 
            el = p + np.random.normal(0, sigma, size=len(p))
            # we should check that values are in appropriate bounds
            if all([left < x < right for (left, right), x in zip(bounds, el)]):
                i += 1
                offsprings.append(el)
    return np.array(offsprings)


In [5]:
def next_gen(pop, offsprings, obj_func, lmd, strategy):
    if strategy == "plus":
        next_generation = np.vstack((pop, offsprings))
    if strategy == "comma":
        next_generation = offsprings
    res = sorted(next_generation, key=obj_func)
    return np.array(res[:lmd])

In [19]:
# the strategy with checking whether the points are not far away from each other
# has not proven to be effective
def should_stop(func, pop, eps = 0.1, rate=0.85):
    scores = [func(p) for p in pop]
    best_score = scores[0]
    diff = [abs(score - best_score) < eps for score in scores]
    return np.count_nonzero(diff) / len(diff) >= rate

## $(\mu, \lambda)$-ES or  $(\mu+\lambda)$-ES

In [7]:
def evolution_strategy(objective, sigma=20, lmd=20, mu=7, strategy="plus", eps=0.1, n_iter=10000, rate=0.5):
    data = functions_data.get(objective)
    if not data:
        return "Unknown function"
    obj_func, bounds = data["func"], data["bounds"]
    population = generate_points(obj_func, bounds, lmd)
    i = 0
    while not should_stop(obj_func, population, eps, rate) and i < n_iter:
        offsprings = get_offsprings(population, sigma, mu, bounds)
        population = next_gen(population, offsprings, obj_func, lmd, strategy)
        i += 1
    return i, population[0], obj_func(population[0])


#### Check strategies for Rastrigin function

In [8]:
func_name, n = "rastrigin", 10
best = functions_data[func_name]["best"]
print(f"Optimum: {best[0]}, value: {best[1]}")

Optimum: [0.0, 0.0], value: 0.0


##### $(\mu + \lambda)$ - ES

In [18]:
n, lmd, mu, strategy, eps = 5, 20, 7, "plus", 0.1

results = []
for sigma in [0.5, 1, np.sqrt(5)]:
    print(f"\u03C3 = {sigma}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")


σ = 0.5
Average   iteration: 169 , avg_point: (-0.0008,  0.0011), avg_value: 0.0100

σ = 1
Average   iteration: 491 , avg_point: (-0.0013, -0.0045), avg_value: 0.0075

σ = 2.23606797749979
Average   iteration: 2294, avg_point: ( 0.0001, -0.0004), avg_value: 0.0040



In [19]:
n, sigma, lmd, strategy, eps = 10, 0.5, 20, "plus", 0.1

results = []
for mu in [3, 7, 15]:
    print(f"\u03BC = {mu}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

μ = 3
Average   iteration: 312 , avg_point: (-0.0011,  0.0003), avg_value: 0.0055

μ = 7
Average   iteration: 138 , avg_point: ( 0.0005, -0.0007), avg_value: 0.0074

μ = 15
Average   iteration: 67  , avg_point: (-0.0008,  0.0004), avg_value: 0.0063



In [20]:
n, sigma, mu, strategy, eps = 10, 0.5, 7, "plus", 0.1

results = []
for lmd in [10, 20, 50]:
    print(f"\u03BB = {lmd}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

λ = 10
Average   iteration: 122 , avg_point: ( 0.0002, -0.0010), avg_value: 0.0082

λ = 20
Average   iteration: 141 , avg_point: (-0.0004,  0.0010), avg_value: 0.0052

λ = 50
Average   iteration: 127 , avg_point: ( 0.0001,  0.0003), avg_value: 0.0025



##### $(\mu, \lambda)$ - ES

In [21]:
n, lmd, mu, strategy, eps, n_iter = 5, 20, 15, "comma", 0.2, 1000

results = []
for sigma in [0.1, 0.3, 0.5]:
    print(f"\u03C3 = {sigma}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, n_iter=n_iter)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")


σ = 0.1
Average   iteration: 6   , avg_point: (-0.5945, -0.0016), avg_value: 1.4055

σ = 0.3
Average   iteration: 1000, avg_point: (-0.0043,  0.0061), avg_value: 0.2138

σ = 0.5
Average   iteration: 1000, avg_point: ( 0.0093, -0.1755), avg_value: 0.6321



In [23]:
n, sigma, lmd, strategy, eps, n_iter = 10, 0.3, 20, "comma", 0.2, 1000

results = []
for mu in [3, 7, 15]:
    print(f"\u03BC = {mu}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, n_iter=n_iter)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

μ = 3
Average   iteration: 1000, avg_point: ( 0.3962, -0.3065), avg_value: 1.8255

μ = 7
Average   iteration: 1000, avg_point: (-0.0005,  0.0900), avg_value: 0.4283

μ = 15
Average   iteration: 1000, avg_point: ( 0.0032, -0.0115), avg_value: 0.0767



In [24]:
n, sigma, mu, strategy, eps, n_iter = 10, 0.3, 15, "comma", 0.2, 1000

results = []
for lmd in [10, 20, 50]:
    print(f"\u03BB = {lmd}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, n_iter=n_iter)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

λ = 10
Average   iteration: 1000, avg_point: (-0.0033,  0.0050), avg_value: 0.1982

λ = 20
Average   iteration: 1000, avg_point: ( 0.0023,  0.0036), avg_value: 0.0945

λ = 50
Average   iteration: 1000, avg_point: ( 0.0009,  0.0054), avg_value: 0.0445



### Check stategies for Booth function

In [8]:
func_name = "booth"
best = functions_data[func_name]["best"]
print(f"Optimum: {best[0]}, value: {best[1]}")

Optimum: [1, 3], value: 0.0


##### $(\mu + \lambda)$ - ES

In [26]:
n, lmd, mu, strategy, eps = 5, 20, 7, "plus", 0.1

results = []
for sigma in [0.5, 1, np.sqrt(5)]:
    print(f"\u03C3 = {sigma}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")


σ = 0.5
Average   iteration: 5   , avg_point: ( 0.9884,  3.0167), avg_value: 0.0042

σ = 1
Average   iteration: 11  , avg_point: ( 0.9983,  3.0052), avg_value: 0.0053

σ = 2.23606797749979
Average   iteration: 35  , avg_point: ( 0.9888,  3.0161), avg_value: 0.0046



In [27]:
n, sigma, lmd, strategy, eps = 10, 0.5, 20, "plus", 0.1

results = []
for mu in [3, 7, 15]:
    print(f"\u03BC = {mu}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

μ = 3
Average   iteration: 12  , avg_point: ( 1.0008,  2.9925), avg_value: 0.0048

μ = 7
Average   iteration: 6   , avg_point: ( 0.9965,  3.0102), avg_value: 0.0051

μ = 15
Average   iteration: 4   , avg_point: ( 1.0173,  2.9910), avg_value: 0.0027



In [28]:
n, sigma, mu, strategy, eps = 10, 0.5, 7, "plus", 0.01

results = []
for lmd in [10, 20, 50]:
    print(f"\u03BB = {lmd}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

λ = 10
Average   iteration: 26  , avg_point: ( 1.0086,  2.9941), avg_value: 0.0016

λ = 20
Average   iteration: 25  , avg_point: ( 1.0060,  2.9951), avg_value: 0.0004

λ = 50
Average   iteration: 21  , avg_point: ( 0.9961,  3.0041), avg_value: 0.0002



##### $(\mu, \lambda)$ - ES

In [9]:
n, lmd, mu, strategy, eps, rate = 5, 20, 7, "comma", 0.1, 0.7

results = []
for sigma in [0.5, 1, np.sqrt(5)]:
    print(f"\u03C3 = {sigma}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, rate=rate)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")


σ = 0.5
Average   iteration: 54  , avg_point: ( 0.9568,  3.0411), avg_value: 0.0128

σ = 1
Average   iteration: 10000, avg_point: ( 1.0246,  2.9524), avg_value: 0.0502

σ = 2.23606797749979
Average   iteration: 10000, avg_point: ( 1.0148,  3.0517), avg_value: 0.0706



In [11]:
n, sigma, lmd, strategy, eps, rate = 10, 0.5, 20, "comma", 0.1, 0.7

results = []
for mu in [3, 7, 15]:
    print(f"\u03BC = {mu}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, rate=rate)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

μ = 3
Average   iteration: 10000, avg_point: ( 0.9782,  3.0084), avg_value: 0.0307

μ = 7
Average   iteration: 42  , avg_point: ( 1.0223,  2.9959), avg_value: 0.0147

μ = 15
Average   iteration: 5   , avg_point: ( 1.0073,  2.9931), avg_value: 0.0047



In [12]:
n, sigma, mu, strategy, eps, rate = 5, 0.5, 7, "comma", 0.1, 0.7

results = []
for lmd in [10, 20, 50]:
    print(f"\u03BB = {lmd}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

λ = 10
Average   iteration: 6   , avg_point: ( 1.0515,  2.9918), avg_value: 0.0421

λ = 20
Average   iteration: 8   , avg_point: ( 0.9389,  3.0336), avg_value: 0.0142

λ = 50
Average   iteration: 15  , avg_point: ( 1.0249,  2.9845), avg_value: 0.0057



### Check strategies for Eggholder function

In [13]:
func_name, n = "eggholder", 10
best = functions_data[func_name]["best"]
print(f"Optimum: {best[0]}, value: {best[1]}")

Optimum: [512, 404.2319], value: -959.6407


##### $(\mu + \lambda)$ - ES

In [25]:
n, lmd, mu, strategy, eps, rate = 5, 500, 7, "plus", 0.01, 0.9

results = []
for sigma in [1, 5, 10]:
    print(f"\u03C3 = {sigma}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, rate=rate)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")


σ = 1
Average   iteration: 29  , avg_point: ( 248.5841,  445.1919), avg_value: -922.2244

σ = 5


KeyboardInterrupt: 

In [22]:
n, sigma, lmd, strategy, eps, rate = 5, 1, 20, "plus", 0.01, 0.95

results = []
for mu in [3, 7, 15]:
    print(f"\u03BC = {mu}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, rate=rate)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

μ = 3
Average   iteration: 37  , avg_point: ( 80.3668,  96.3883), avg_value: -688.1911

μ = 7
Average   iteration: 198 , avg_point: ( 25.8729,  310.4048), avg_value: -768.1924

μ = 15
Average   iteration: 23  , avg_point: ( 260.5391, -26.1947), avg_value: -795.7831



In [20]:
n, sigma, mu, strategy, eps, rate = 5, 1, 7, "plus", 0.1, 0.9

results = []
for lmd in [100, 200, 500]:
    print(f"\u03BB = {lmd}")
    avg_iter, avg_point, avg_score = 0, np.zeros((2,)), 0
    for i in range(n):
        res = evolution_strategy(func_name, sigma=sigma, lmd=lmd, mu=mu, strategy=strategy, eps=eps, rate=rate)
        results.append(res)
        avg_iter += res[0]
        avg_point += res[1]
        avg_score += res[2]
#         print(f"iteration: {res[0]:<4d}, point: ({res[1][0]: .4f}, {res[1][1]: .4f}), value: {res[2]:<.4f}")
    print(f"Average   iteration: {int(avg_iter / n):<4d}, avg_point: ({avg_point[0] / n: .4f}, {avg_point[1] / n: .4f}), avg_value: {avg_score / n:<.4f}\n")

λ = 100
Average   iteration: 22  , avg_point: ( 436.1428,  284.5372), avg_value: -891.5019

λ = 200
Average   iteration: 11  , avg_point: (-134.9077,  224.8057), avg_value: -867.4483

λ = 500
Average   iteration: 17  , avg_point: ( 456.6367,  445.5422), avg_value: -943.9700

